In [1]:
import openmc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob


def run_simulation(
    fuente: list,
    geometria: list,
    z0: float,
    z_track: float,
    N_particles: int,
    ww: bool,
) -> None:
    """
    Ejecuta una simulación en OpenMC con la configuración especificada.

    Parámetros:
        fuente (list):
            - Si tiene 1 elemento: se asume que es la ruta a un archivo de fuente.
            - Si tiene 2 elementos: se asume que es [fuente_energia, fuente_direccion] para fuente independiente.
                - fuente_energia puede ser:
                    - "monoenergetica": Fuente con energía fija.
                    - "espectro_fision": Fuente con espectro de fisión.
                    - "espectro_termico": Fuente con espectro térmico.
                - fuente_direccion puede ser:
                    - "colimada": Fuente con dirección fija.
                    - "isotropica": Fuente con distribución isotrópica.

        geometria (list): Parámetros geométricos:
            - geometria[0] (bool): Flag para indicar si existe región de vacío.
            - geometria[1] (float): L_x, ancho del paralelepípedo.
            - geometria[2] (float): L_y, ancho del paralelepípedo.
            - geometria[3] (float): L_z, altura del paralelepípedo.
            - Si geometria[0] es True, se esperan además:
                - geometria[4] (float): L_x_vacio.
                - geometria[5] (float): L_y_vacio.

        z0 (float): Posición en z donde empieza la simulación.
        z_track (float): Posición en z para la superficie de track.
        N_particles (int): Número de partículas a simular en total.
        outfile_name (str): Nombre del archivo de salida. Si es None, se usa el nombre por defecto.
        WW (bool): Si se activa, se generan weight windows.
    """
    # ---------------------------------------------------------------------------------------------------------------------------------------
    # Configuración de secciones eficaces
    # ---------------------------------------------------------------------------------------------------------------------------------------
    openmc.config["cross_sections"] = (
        "/home/lucas/Documents/Proyecto_Integrador/endfb-viii.0-hdf5/cross_sections.xml"
    )

    # ----------------------------------------------------------------------------------------------------------------------------------------
    # Procesamiento de la fuente
    # ----------------------------------------------------------------------------------------------------------------------------------------
    # Si la longitud de la fuente es 1, se asume que es un archivo de fuente.
    # Si la longitud de la fuente es 2, se asume que es una fuente independiente.
    if len(fuente) == 1:
        # Procesar fuente cuando viene de archivo
        ruta_fuente = fuente[0]
        if ruta_fuente.lower().endswith(".xml"):
            # Si es un XML de OpenMC, leerlo con el parser de fuentes
            source = openmc.HistogramSource(
                path=ruta_fuente,
            )
        else:
            # Si no es XML, asumimos formato binario o HDF5 compatible con FileSource
            source = openmc.FileSource(ruta_fuente)
    elif len(fuente) == 2:
        fuente_energia, fuente_direccion = fuente
        source = openmc.IndependentSource()
        source.particle = "neutron"

        # Distribución espacial: se coloca en la región central de la geometría
        L_x, L_y = geometria[1], geometria[2]
        x_dist = openmc.stats.Uniform(-L_x / 2, L_x / 2)
        y_dist = openmc.stats.Uniform(-L_y / 2, L_y / 2)
        z_dist = openmc.stats.Discrete(z0 + 1e-6, 1)  # Se fija z muy cerca de z=0
        source.space = openmc.stats.CartesianIndependent(x_dist, y_dist, z_dist)

        # Distribución de energía
        if fuente_energia == "monoenergetica":
            source.energy = openmc.stats.Discrete([1e6], [1])

        # Distribución angular
        if fuente_direccion == "colimada":
            mu = openmc.stats.Discrete([1], [1])
            phi = openmc.stats.Uniform(0.0, 2 * np.pi)
            source.angle = openmc.stats.PolarAzimuthal(mu, phi)
    else:
        raise ValueError("El parámetro 'fuente' debe contener 1 o 2 elementos.")

    # ---------------------------------------------------------------------------------------------------------------------------------------
    # Procesamiento de la geometría
    # ---------------------------------------------------------------------------------------------------------------------------------------
    # Extraer parámetros geométricos
    vacio = geometria[0]
    L_x, L_y, L_z = geometria[1:4]
    if vacio:
        L_x_vacio, L_y_vacio = geometria[4:6]

    # Definir material: agua
    mat_agua = openmc.Material(name="agua")
    mat_agua.add_nuclide("H1", 2.0, "ao")
    mat_agua.add_nuclide("O16", 1.0, "ao")
    mat_agua.add_s_alpha_beta("c_H_in_H2O")
    mat_agua.set_density("g/cm3", 1.0)
    mats = openmc.Materials([mat_agua])
    mats.export_to_xml()

    # Definir superficies externas
    surfaces = {
        "x_min": openmc.XPlane(x0=-L_x / 2, boundary_type="vacuum"),
        "x_max": openmc.XPlane(x0=L_x / 2, boundary_type="vacuum"),
        "y_min": openmc.YPlane(y0=-L_y / 2, boundary_type="vacuum"),
        "y_max": openmc.YPlane(y0=L_y / 2, boundary_type="vacuum"),
        "z_min": openmc.ZPlane(z0=z0, boundary_type="vacuum"),
        "z_max": openmc.ZPlane(z0=L_z, boundary_type="vacuum"),
    }

    # Se agrega la superficie de registro para generar el track file
    if z_track is not None:
        surfaces.update(
            {
                "z_track": openmc.ZPlane(
                    z0=z_track, boundary_type="transmission", surface_id=70
                )
            }
        )

    # Si hay vacío, definir superficies internas
    if vacio:
        surfaces.update(
            {
                "x_min_vacio": openmc.XPlane(
                    x0=-L_x_vacio / 2, boundary_type="transmission"
                ),
                "x_max_vacio": openmc.XPlane(
                    x0=L_x_vacio / 2, boundary_type="transmission"
                ),
                "y_min_vacio": openmc.YPlane(
                    y0=-L_y_vacio / 2, boundary_type="transmission"
                ),
                "y_max_vacio": openmc.YPlane(
                    y0=L_y_vacio / 2, boundary_type="transmission"
                ),
            }
        )

    # Para fuente tipo FileSource se traduce la superficie inferior para posicionar z0.
    # Sino se hace entonces las particulas aparecer fuera de la geometria.
    if len(fuente) == 1:
        surfaces["z_min"].translate(vector=(0, 0, -1e-6), inplace=True)

    # Definir regiones
    region_externa = (
        +surfaces["x_min"]
        & -surfaces["x_max"]
        & +surfaces["y_min"]
        & -surfaces["y_max"]
        & +surfaces["z_min"]
        & -surfaces["z_max"]
    )

    if vacio:
        region_vacio = (
            +surfaces["x_min_vacio"]
            & -surfaces["x_max_vacio"]
            & +surfaces["y_min_vacio"]
            & -surfaces["y_max_vacio"]
            & +surfaces["z_min"]
            & -surfaces["z_max"]
        )

    # Crear universo y definir celdas según configuración de fuente y vacío
    universe = openmc.Universe()

    if vacio:
        if z_track is not None:
            universe.add_cell(
                openmc.Cell(
                    region=region_externa & ~region_vacio & -surfaces["z_track"],
                    fill=mat_agua,
                    name="agua1",
                )
            )
            universe.add_cell(
                openmc.Cell(
                    region=region_externa & ~region_vacio & +surfaces["z_track"],
                    fill=mat_agua,
                    name="agua2",
                )
            )
            universe.add_cell(
                openmc.Cell(
                    region=region_vacio & -surfaces["z_track"],
                    fill=None,
                    name="vacio1",
                )
            )
            universe.add_cell(
                openmc.Cell(
                    region=region_vacio & +surfaces["z_track"],
                    fill=None,
                    name="vacio2",
                )
            )
        else:
            universe.add_cell(
                openmc.Cell(
                    region=region_externa & ~region_vacio,
                    fill=mat_agua,
                    name="agua",
                )
            )
            universe.add_cell(openmc.Cell(region=region_vacio, fill=None, name="vacio"))
    else:
        if z_track is not None:
            universe.add_cell(
                openmc.Cell(
                    region=region_externa & -surfaces["z_track"],
                    fill=mat_agua,
                    name="agua1",
                )
            )
            universe.add_cell(
                openmc.Cell(
                    region=region_externa & +surfaces["z_track"],
                    fill=mat_agua,
                    name="agua2",
                )
            )
        else:
            universe.add_cell(
                openmc.Cell(region=region_externa, fill=mat_agua, name="agua")
            )

    geom = openmc.Geometry(universe)
    geom.export_to_xml()

    # ---------------------------------------------------------------------------------------------------------------------------------------
    # Configuración de settings
    # ---------------------------------------------------------------------------------------------------------------------------------------
    settings = openmc.Settings()
    settings.surf_source_write = {"surface_ids": [70], "max_particles": 20000000}
    settings.run_mode = "fixed source"
    settings.batches = 1000
    settings.particles = int(N_particles / 1000)
    settings.source = source

    # Se definen las ventanas de peso
    if ww:
        # Define weight window spatial mesh
        ww_mesh = openmc.RegularMesh()
        ww_mesh.dimension = (10, 10, 100)
        ww_mesh.lower_left = (-L_x / 2, -L_y / 2, z0)
        ww_mesh.upper_right = (L_x / 2, L_y / 2, L_z)

        # Create weight window object and adjust parameters
        wwg = openmc.WeightWindowGenerator(
            method="magic",
            mesh=ww_mesh,
            max_realizations=settings.batches,
            energy_bounds=[0, 1, 1e3, 1e7],
            update_interval=50,
        )

        # Add generator to Settings instance
        settings.weight_window_generators = wwg
    settings.export_to_xml()

    # ---------------------------------------------------------------------------------------------------------------------------------------
    # Configuración de tallies
    # ---------------------------------------------------------------------------------------------------------------------------------------
    # Tally: malla para flujo total
    mesh = openmc.RectilinearMesh()
    mesh.x_grid = np.linspace(-L_x / 2, L_x / 2, 2)
    mesh.y_grid = np.linspace(-L_y / 2, L_y / 2, 2)
    mesh.z_grid = np.linspace(z0, L_z, 51)
    mesh_filter = openmc.MeshFilter(mesh)
    tally_flux_total = openmc.Tally(name="flux_total")
    tally_flux_total.filters = [mesh_filter]
    tally_flux_total.scores = ["flux"]

    # Tally: malla para flujo en vacio
    if vacio:
        mesh_vacio = openmc.RectilinearMesh()
        mesh_vacio.x_grid = np.linspace(-L_x_vacio / 2, L_x_vacio / 2, 2)
        mesh_vacio.y_grid = np.linspace(-L_y_vacio / 2, L_y_vacio / 2, 2)
        mesh_vacio.z_grid = np.linspace(z0, L_z, 51)
        mesh_filter_vacio = openmc.MeshFilter(mesh_vacio)
        tally_flux_vacio = openmc.Tally(name="flux_vacio")
        tally_flux_vacio.filters = [mesh_filter_vacio]
        tally_flux_vacio.scores = ["flux"]

    tallies = openmc.Tallies(
        [tally_flux_total, tally_flux_vacio] if vacio else [tally_flux_total]
    )

    tallies.export_to_xml()

    # ---------------------------------------------------------------------------------------------------------------------------------------
    # Limpieza de archivos previos y ejecución de la simulación
    # ---------------------------------------------------------------------------------------------------------------------------------------
    for file in glob.glob("statepoint.*.h5"):
        os.remove(file)
    if os.path.exists("summary.h5"):
        os.remove("summary.h5")

    openmc.run()

In [2]:
geometria = [True, 15, 15, 300, 3, 3]
z0 = 30
z_track = 60
fuente = ["/home/lucas/Documents/Proyecto_Integrador/PI/resampleo_trackfiles/trackfile4/config1/source.xml"]
num_particles = int(5e3)
WW = True

# Ejecutar la simulación
run_simulation(
    fuente=fuente,
    geometria=geometria,
    z0=z0,
    z_track=z_track,
    N_particles=num_particles,
    ww=WW,
)

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

KeyboardInterrupt: 